In [1]:
import pandas as pd
import numpy as np
import re

Вместо твитов про ковид будем использовать твиты известного политика, так как иначе не хватает ОЗУ в Google Colab

In [35]:
tweets = pd.read_csv('https://raw.githubusercontent.com/evlko/CS-224W/main/Data/Twitter/tweets_DT.csv')
tweets = tweets.rename(columns={'Tweet_Text': 'text'})

Дополнительно возьмем лишь часть из них:

In [36]:
tweets = tweets.head(1000)

# Preprocessing
Важно, что так как мы будем решать задачу генерации текста, а не классификации, то:
1. удалять стоп-слова не надо, так как без них будет потеряна логика языка;
2. лематизировать не имеет смысла, так как вновь потеряется смысл.

In [4]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt', quiet=True)

True

Функция для удаления слов, которые начинаются с определенных символов

In [5]:
def remove_words_with_sym(text, symbol='#'):
  return re.sub(r'{}[^\s]*'.format(symbol), '', text)

assert remove_words_with_sym('Hello, it\'s a me, Mario!, #Top') == 'Hello, it\'s a me, Mario!, '
assert remove_words_with_sym('Hello, it\'s a me, @Mario!', '@') == 'Hello, it\'s a me, '

Полная функция для предобработки

In [6]:
def preprocess(text, special_symbols=['#', '@', 'http', 'pic twitter c', 'rt']):
    text = re.sub(r'[^a-zA-Z\s]', '', text).replace('_', '')
    text = text.lower()
    for special_symbol in special_symbols:
      text = remove_words_with_sym(text, special_symbol)
    text = [word for word in word_tokenize(text)] 
    text = ' '.join(text)
    return text

assert preprocess('Hello, it\'s a me, Mario! 999') == 'hello its a me mario'

In [37]:
tweets['text'] = tweets['text'].apply(preprocess)

# EDA
Проведем небольшой Exploratory Data Analysis. Узнаем:
1. Минимальную длину твита;
2. Среднюю;
3. Максимальную.
По итогу будем стараться генерировать средние твиты.

In [38]:
min_tweet_len, avg_tweet_len, max_tweet_len = float('inf'), 0, 0

for tweet in tweets['text'].tolist():
  tweet_len = len(tweet.split())
  if tweet_len < min_tweet_len:
    min_tweet_len = tweet_len
  elif tweet_len > max_tweet_len:
    max_tweet_len = tweet_len
  avg_tweet_len += tweet_len
avg_tweet_len = avg_tweet_len // (len(tweets))

min_tweet_len, avg_tweet_len, max_tweet_len

(1, 15, 29)

# Tokenizing

Посчитаем число уникальных токенов, а также сохраним их

In [39]:
text_words = word_tokenize(' '.join(tweets['text'].to_numpy()))
n_words = len(text_words)
unique_words = set(text_words)

print('Total Words: %d' % n_words)
print('Unique Words: %d' % len(unique_words))

Total Words: 15259
Unique Words: 2850


Создаем словарь, где ключ — слово, а значение — целое число. 

На будущее создадим "обратный" словарь. Проблем не будет, так как в данном случае мы имеем биективное отношение.

In [40]:
word_2_index = { w: i for i, w in enumerate(unique_words) }
index_2_word = { i: w for i, w in enumerate(unique_words) }

## Подготовка Данных
Задача генерации — предсказать следующее слово по текущим, то есть у нас уже есть некоторая последовательность слов и мы пытаемся угадать следующее слово. Разобьем наш текст на последовательности определенной длиный и будем запоминать данные.

Очевидно, что у такого подхода есть 2 проблемы:
1. Весь текст был объединен в одну цепочку, то есть в какой-то момент твиты идут друг за другом, что нелогично, поэтому имеет смысл оперировать не всем набором слов, а отдельными твитами, каждый из которых и пытается сгенерировать свою цепочку;
2. Подбор длины - отдельная задача, которую надо решать.

In [134]:
from keras.utils import to_categorical

In [41]:
input_sequence = []
output_words = []
input_seq_length = avg_tweet_len-1

tweets_texts = tweets['text']

for tweet in tweets_texts:
  tokenized_tweet = tweet.split()
  if len(tokenized_tweet) < input_seq_length:
    continue
  for i in range(0, len(tokenized_tweet) - input_seq_length):
    in_seq = tokenized_tweet[i:i + input_seq_length]
    out_seq = tokenized_tweet[i + input_seq_length]
    input_sequence.append([word_2_index[word] for word in in_seq])
    output_words.append(word_2_index[out_seq])

In [42]:
X = np.reshape(input_sequence, (len(input_sequence), input_seq_length, 1))

y = to_categorical(output_words)

In [43]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (3308, 14, 1)
y shape: (3308, 2850)


# LSTM

Создадим простую модель, которая будет состоять из нескольких `LSTM` слоев и дополнительных-инструментальных (`Dense`, `Dropout`, etc)

In [44]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Embedding, Dropout

In [56]:
model = Sequential()
model.add(LSTM(300, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(100))
model.add(Dense(y.shape[1], activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 14, 300)           362400    
                                                                 
 dropout_3 (Dropout)         (None, 14, 300)           0         
                                                                 
 lstm_7 (LSTM)               (None, 100)               160400    
                                                                 
 dense_3 (Dense)             (None, 2850)              287850    
                                                                 
Total params: 810,650
Trainable params: 810,650
Non-trainable params: 0
_________________________________________________________________


Обучаем модель:

In [ ]:
model.fit(X, y, epochs=100, verbose=5)

## Генерация

Выберем случайную последовательность:

In [131]:
random_seq_index = np.random.randint(0, len(input_sequence)-1)
random_seq = input_sequence[random_seq_index]
word_sequence = [index_2_word[value] for value in random_seq]

' '.join(word_sequence)

'come at you from all sides they dont know how to win i will'

Попробуем продолжить твит до максимальной длины: 

In [132]:
for i in range(max_tweet_len-len(word_sequence)+1):
  int_sample = np.reshape(random_seq, (1, len(random_seq), 1))

  predicted_words_indices = model.predict(int_sample, verbose=5)
  predicted_word_index = np.argmax(predicted_words_indices)

  random_seq.append(predicted_word_index)
  random_seq = random_seq[1:len(random_seq)]

  word_sequence.append(index_2_word[predicted_word_index])

Результат генерации

In [133]:
' '.join(word_sequence)

'come at you from all sides they dont know how to win i will morningjoe apologize and never statistics doubt defending will make d loyal the any even oh totally'